In [297]:
# Most Recent One (Suggested)
#!pip install git+https://github.com/rcmalli/keras-vggface.git
# Release Version
#!pip install keras_vggface

In [298]:
#!pip uninstall tensorflow --y
#!pip install --upgrade tensorflow


In [299]:

# confirm mtcnn was installed correctly
import mtcnn
# print version
print(mtcnn.__version__)

0.1.0


In [300]:
import os
list_of_images=os.listdir('/Users/Guchi/Documents/projects/face detection model/images')


In [301]:
images_dict={i:file for i,file in enumerate(list_of_images)}

In [302]:
import matplotlib.pyplot as plt


In [303]:
import cv2

In [304]:
import numpy as np
from mtcnn import MTCNN

In [305]:
extracted_images_dic1={}
def extract_face(filename, required_size=(224, 224)):
    filename_='images/'+str(filename)
    # load image from file
    pixels = plt.imread(filename_)
    # create the detector, using default weights
    detector = MTCNN()
    # detect faces in the image
    results = detector.detect_faces(pixels)
    # extract the bounding box from the first face
    extracted_images=[]
    for i in range(0,len(results)):
        x1, y1, width, height = results[i]['box']
        x2, y2 = x1 + width, y1 + height
        face = pixels[y1:y2, x1:x2]
        image=cv2.resize(face,required_size)
        extracted_images.append(image)
        extracted_images_dic={filename:extracted_images}
        extracted_images_dic1.update(extracted_images_dic)

   
    return extracted_images_dic1


In [306]:
imc=list(map(extract_face,list_of_images))
imc=imc[0]

1/1 [==============================] - 0s 93ms/step


In [307]:
import keras
import torch
from facenet_pytorch import InceptionResnetV1

In [308]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [309]:
model=InceptionResnetV1(pretrained='vggface2').eval().to(device)

In [310]:
def generate_embedding(pixels):

  emb_list=[]
  name_list=[]
  for v,i in pixels.items():
    for sep in i:
      face=torch.tensor(sep).to(device)
      face=(face/255.0-0.5)/.5
      face=face.unsqueeze(0)
      tensor_permuted = face.permute(0, 3, 1, 2)
      embeddings=model(tensor_permuted)
      emb_list.append(embeddings)
      name_list.append(v)


  print(embeddings.shape)
  return emb_list,name_list



In [311]:
embeddings,list_names=generate_embedding(imc)

torch.Size([1, 512])


In [312]:

embed_name_dic={i:v.detach().numpy() for v,i in zip(embeddings,list_names)}

In [313]:
def stack_embed(embeddings):
    stacked_embeddings = torch.cat(embeddings, dim=0)
    return stacked_embeddings

In [314]:
embeddd=stack_embed(embeddings)


In [315]:
final_embeddings=embeddd

In [316]:
final_embeddings.shape

torch.Size([3, 512])

In [317]:
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd 

In [318]:
type(final_embeddings)

torch.Tensor

In [319]:

result_indices = []
for i in range(final_embeddings.shape[0]):
    score = torch.nn.functional.cosine_similarity(final_embeddings[i], final_embeddings)
    indices = np.where(score > 0.8)[0] 
    result_indices.append(indices)
    print(score)
    
    



tensor([1.0000, 0.8826, 0.2129], grad_fn=<SumBackward1>)
tensor([0.8826, 1.0000, 0.1810], grad_fn=<SumBackward1>)
tensor([0.2129, 0.1810, 1.0000], grad_fn=<SumBackward1>)


In [320]:
result_indices

[array([0, 1]), array([0, 1]), array([2])]

In [321]:
xx=pd.Series([list(i) for i in result_indices if len(i)>1]).drop_duplicates().reset_index(drop=True)
xx

0    [0, 1]
dtype: object

In [322]:
os.getcwd()

'/Users/Guchi/Documents/projects/face detection model'

In [323]:
for m,i in enumerate(xx):
    os.makedirs(str(m),exist_ok=True)
    for j in i:
        k=list_names[j]
        l=f'./images/{k}'
        imagee=plt.imread(l)
        plt.imsave(f'./{m}/{k}',imagee)

 
